In [1]:
# -*- coding: utf-8 -*-
import jieba, gensim, re, sqlite3, os
import jieba.posseg as pseg
import codecs
from hanziconv import HanziConv
import numpy as np


In [2]:
# dump sql to raw txt

# conn = sqlite3.connect('apple.sqlite3')
# cursor = conn.execute("SELECT content from apple")
# removetokens = [u'翻攝畫面']
# def get_sentences(c):
#     i = 0
#     for w in c:
#         context = w[0]
#         context = re.sub(ur'[^\u4e00-\u9fff]+', ' ', context)
#         context = HanziConv.toSimplified(context)
#         for r in removetokens:
#             context = re.sub(r, ' ', context)
#         sentences = context.split(' ')
#         for s in sentences:
#             if len(s) > 10:
#                 temp = pseg.cut(s)
#                 words = []
#                 semantics = []
#                 for t in temp:
#                     words.append(t.word)
#                     semantics.append(t.flag)
#                 yield '/'.join(words)+ '>>>' + '/'.join(semantics)

#     c.close()
# sentences = get_sentences(cursor)
# with codecs.open('sentences.txt', 'w', encoding='utf8') as f:
#     for s in sentences:
#         f.write(s+'\n')
# cursor.close()
# conn.close()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/4_/b_9vfbwx41g4t48jrzr0r4vm0000gn/T/jieba.cache
Loading model cost 0.465 seconds.
Prefix dict has been built succesfully.


In [12]:
# extract semantic representation

# with codecs.open('semantics.txt', 'w', encoding='utf8') as g:
#     with codecs.open('sentences.txt', 'r', encoding='utf8') as f:
#         for line in f.readlines():
#             idx = line.rfind('>>>')
#             target = line[idx + 3:]
#             g.write(target)



In [2]:
# train word2vec model
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            with open(self.dirname + '/' + fname, 'r') as f:
                for line in f.readlines():
                    sen = line[:-1].split('/')
                    yield sen

# print os.listdir('testdir')
sentences = MySentences('testdir')
# i = 0
# for s in sentences:
#     print s
#     if i > 10:
#         break
model = gensim.models.Word2Vec(sentences, window = 5, min_count = 5, size = 30, iter = 500)


In [3]:
# debug
print len(model.vocab)

51


In [4]:
# make pseudo cloze quiz for validation
import random

def make_cloze(num):
    gidx = range(0, 20178)
    random.shuffle(gidx)
    gidx = sorted(set(gidx[: num]))
    
    curr_idx = 0
    sentences = []
    with open('testdir/semantics.txt', 'r') as f:
        for i in gidx:
            for j in xrange(i - curr_idx): f.next()
            sentences.append(f.next().split('/')[:-1])
            curr_idx = i + 1
     
    options = []
    keys = model.vocab.keys()
    for j in xrange(num):
        l = len(sentences[j])
        if(l <= 0):
            print gidx[j]
        idx = range(0, 51)
        random.shuffle(idx)
        
        candidate = [keys[i] for i in idx[:3]]
#         count = 0
#         while count < 10:
        i = random.randint(0, l - 1)
#             if len(s[i]) > 4 or count == 9:
        options.append((i, [sentences[j][i]] + candidate))
#                 break
#             count += 1
    return sentences, options

sentences, options = make_cloze(3)
for i in xrange(len(sentences)):
    print sentences[i]
    print options[i]
    print '---------------------------------------------'

['zg', 'n', 'n', 'n', 'n']
(3, ['n', 'x', 'uj', 'ns'])
---------------------------------------------
['t', 'v', 'p', 'vn', 'v', 'v', 'n', 'b']
(7, ['b', 'nrt', 'vn', 'an'])
---------------------------------------------
['t', 'd', 'v', 'v', 'a', 'x', 'x', 'uj', 'n']
(2, ['v', 'mq', 'd', 'n'])
---------------------------------------------


In [5]:
# cloze test method
def cloze_test(string, index, candidates):
    temp = string[:]
    del temp[index]
    ref_vec = np.zeros(30)
    l = 0
    for w in temp:
        if w in model.vocab:
            ref_vec += model[w]
        l += 1
            
#     ref_vec /= float(l)
#     ref_vec = ref_vec / np.linalg.norm(ref_vec)
    nc = len(candidates)
    
    arr = np.zeros(nc)
    for i in xrange(nc):
        if candidates[i] in model.vocab:
            cand_vec = model[candidates[i]]
#             arr[i] = (ref_vec * cand_vec).sum()
            arr[i] = (ref_vec * (cand_vec / np.linalg.norm(cand_vec))).sum()
#     l, r = arr[:, j].min(), arr[:, j].max()
#     if r > l:
#         arr[:, j] = (arr[:, j] - l) / (r - l)
    return arr


def cal_score(sentences, options):
    n = len(sentences)
    s = 0.
    ref = 0.
    for i in xrange(n):
#         print sentences[i]
#         print options[i][0], options[i][1]
        b = cloze_test(sentences[i], options[i][0], options[i][1])
        if b.argmax() == 0: s += 1.
        if random.randint(0, 3) == 0: ref += 1.
    return s/float(n), ref/float(n)

In [7]:
for i in xrange(10):
    sentences, options = make_cloze(2000)
    score, ref = cal_score(sentences, options)
    print score, ',', ref

0.578 , 0.2595
0.5995 , 0.2415
0.583 , 0.251
0.5835 , 0.2505
0.5775 , 0.243
0.584 , 0.2565
0.574 , 0.2585
0.573 , 0.269
0.595 , 0.2485
0.562 , 0.2635


In [37]:
model.save('w2v_apple')

In [4]:
model = gensim.models.Word2Vec.load('w2v_apple')

In [61]:
print range(0, 10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
# s= u'有/在/附近/海域/作业/的/渔民/声称>>>v/p/f/n/n/uj/n/n'
# idx = s.rfind('>>>')
# t = s[idx + 3:]
# print t

# string = '越南空軍一架俄羅斯製造的蘇愷'
# words = pseg.cut(string)
# t = []
# for w in words:
#     t.append(w.word)
# print '/'.join(t)
#     print('%s %s' % (w.word, w.flag))